In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("ProcessedData/NYPD_Motor_Vehicle_Collisions.csv")
# print df

In [7]:
df.columns

Index([u'DATE', u'TIME', u'BOROUGH', u'ZIP CODE', u'LATITUDE', u'LONGITUDE',
       u'ON STREET NAME', u'CROSS STREET NAME', u'OFF STREET NAME',
       u'NUMBER OF PERSONS INJURED', u'NUMBER OF PERSONS KILLED',
       u'NUMBER OF PEDESTRIANS INJURED', u'NUMBER OF PEDESTRIANS KILLED',
       u'NUMBER OF CYCLIST INJURED', u'NUMBER OF CYCLIST KILLED',
       u'NUMBER OF MOTORIST INJURED', u'NUMBER OF MOTORIST KILLED',
       u'CONTRIBUTING FACTOR VEHICLE 1', u'CONTRIBUTING FACTOR VEHICLE 2',
       u'CONTRIBUTING FACTOR VEHICLE 3', u'CONTRIBUTING FACTOR VEHICLE 4',
       u'CONTRIBUTING FACTOR VEHICLE 5', u'VEHICLE TYPE CODE 1',
       u'VEHICLE TYPE CODE 2', u'VEHICLE TYPE CODE 3', u'VEHICLE TYPE CODE 4',
       u'VEHICLE TYPE CODE 5'],
      dtype='object')

In [8]:
processedDf = pd.DataFrame()
processedDf['Borough'] = df['BOROUGH']
processedDf['Zip'] = df['ZIP CODE']
processedDf['latitude'] = df['LATITUDE']
processedDf['longitude'] = df['LONGITUDE']
processedDf['on street name'] = df['ON STREET NAME']
processedDf['cross street name'] = df['CROSS STREET NAME']
processedDf['off street name'] = df['OFF STREET NAME']

In [9]:
processedDf
trainset = processedDf.dropna()
testset = processedDf[processedDf['Borough'].isnull()]
#testset = testset[testset['latitude'].isnull()]
testset.columns

# def predictBorough(x):
#     if(x['latitude'] != None && x['longitude'] != None):
#         #predict based on latitude and longitude
#     elif(x['Zip'] != None):
#         #predict borough based on zip
#     elif(x['off street name']):

Index([u'Borough', u'Zip', u'latitude', u'longitude', u'on street name',
       u'cross street name', u'off street name'],
      dtype='object')

In [10]:
len(trainset.index)

58594

In [11]:
len(testset.index)

298125

In [20]:
from geopy.geocoders import GoogleV3 as g
import time

# geolocator = g()
# location = geolocator.reverse("40.66523, -32.7685", exactly_one = True)
# print(location.address)

def getAddress(row):
    geolocator = g()
    borough = None
    if(not pd.isnull(row['latitude']) and not pd.isnull(row['longitude'])):
#         time.sleep(.5)
        location = geolocator.reverse(str(row['latitude']) + "," + str(row['longitude']), exactly_one = True)
#         print(location)

        if(not location.address is None):
            locationList = location.address.split(",")
            for loc in locationList:
                if "New York County" in loc:
                    borough = "MANHATTAN"
                    break
                elif "Bronx" in loc:
                    borough = "BRONX"
                    break
                elif "Kings" in loc:
                    borough = "BROOKLYN"
                    break
                elif "Queens" in loc:
                    borough = "QUEENS"
                    break
    #             if "County" in loc:
    #                 borough = getBorough(loc.strip())
    
    row['Borough'] = borough
    return borough

def getBorough(county):
    countyToBorough = {"New York County": "MANHATTAN", "Bronx County": "BRONX", "Kings County": "BROOKLYN", "Queens County": "QUEENS", "Richmond County": "STATEN ISLAND"}
    return countyToBorough[county]

numRows = len(testset.index)
subsetNum = 625
subsetSize = numRows/subsetNum

for i in range(1,subsetNum + 1):
    print(str(i))
    testset1 = testset[(i-1)*subsetSize:i*subsetSize]
    testset1 = testset1.apply(getAddress, axis = 1)
    testset[(i-1)*subsetSize:i*subsetSize] = testset1
    testset.to_csv('ProcessedData/collisions/' + 'collisions' + str(i) + '.csv')
    print(testset1)
    
# testset = testset.apply(getAddress, axis = 1)
print(testset)

GeocoderQuotaExceeded: ('The given key has gone over the requests limit in the 24 hour period or has submitted too many requests in too short a period of time.', u'occurred at index 67')